<a href="https://colab.research.google.com/github/NaveenDivakar/Sentiment-Analysis/blob/main/flipkartwebscrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
from urllib.parse import urlparse, parse_qs
import os

# Set up Chrome options
options = Options()
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Automatically find the ChromeDriver executable
# This will handle the path issue if chromedriver is installed in a standard location
driver = webdriver.Chrome(options=options)

# ... (rest of your code) ...
# List of URLs for different phones
phone_urls = {
    "Samsung Galaxy S23": "https://www.flipkart.com/samsung-galaxy-s23-5g-phantom-black-128-gb/product-reviews/itm1f3efe01d1c61?pid=MOBGNPGZVX4PCTTF&lid=LSTMOBGNPGZVX4PCTTFYWYWBL&marketplace=FLIPKART",
    "Vivo T3": "https://www.flipkart.com/vivo-t3-ultra-frost-green-256-gb/product-reviews/itme360ff5b7dbab?pid=MOBH4EACZ7SACMMM&lid=LSTMOBH4EACZ7SACMMMMVZG93&marketplace=FLIPKART",
    "Google Pixel 8": "https://www.flipkart.com/google-pixel-8-hazel-128-gb/product-reviews/itm67e2a2531aaac?pid=MOBGT5F2WD8HPTPZ&lid=LSTMOBGT5F2WD8HPTPZ4A9QHI&marketplace=FLIPKART",
    "Motorola Edge 50": "https://www.flipkart.com/motorola-edge-50-neo-pantone-grisaille-256-gb/product-reviews/itm5b85defa76389?pid=MOBHFHDRVUDB3HFX&lid=LSTMOBHFHDRVUDB3HFXLMEVTW&marketplace=FLIPKART",
    "Realme 12 Pro": "https://www.flipkart.com/realme-12-pro-5g-navigator-beige-256-gb/product-reviews/itmcc78f150eeabd?pid=MOBGYQ6BVDHRJRSG&lid=LSTMOBGYQ6BVDHRJRSGDKJZZD&marketplace=FLIPKART"
}

def fetch_html(url):
    """Fetch the HTML content of a given URL."""
    try:
        driver.get(url)
        time.sleep(2)  # Allow time for the page to load
        return BeautifulSoup(driver.page_source, "html.parser")
    except Exception as e:
        print(f"Error fetching the page: {e}")
        return None


def extract_reviews(soup):
    """Extract reviews from the parsed HTML page."""
    reviews = []
    if not soup:
        return reviews  # Return empty list if no content

    review_blocks = soup.find_all('div', {'class': 'cPHDOP col-12-12'})  # Updated class if structure changes
    for block in review_blocks:
        try:
            rating_elem = block.find('div', {'class': 'XQDdHH Ga3i8K'})
            review_elem = block.find('div', {'class': 'ZmyHeo'})


            if rating_elem and review_elem:
                review = {
                    'Rating': rating_elem.text.strip(),
                    'Review': review_elem.text.strip().replace('READ MORE', ''),

                }
                reviews.append(review)
        except Exception as e:
            print(f"Error parsing a review block: {e}")
    return reviews
def extract_product_id(url):
    """Extract the product ID from the Flipkart URL."""
    parsed_url = urlparse(url)
    product_id = parse_qs(parsed_url.query).get('pid', [None])[0]
    return product_id

# Function to scrape reviews for a single phone
def scrape_reviews_for_phone(phone_name, phone_url):
    reviews = []
    product_id = extract_product_id(phone_url)
    page = 1
    print(f"Scraping reviews for: {phone_name} (Product ID: {product_id})")

    try:
        while True:
            page_url = f"{phone_url}&page={page}"
            print(f"Scraping page {page}...")
            soup = fetch_html(page_url)
            page_reviews = extract_reviews(soup)
            if not page_reviews:
                print(f"No more reviews found for {phone_name} or page structure changed.")
                break
            for review_data in page_reviews:
                # Add product ID and phone name to each review entry
                review_data['Product ID'] = product_id
                review_data['Phone Name'] = phone_name
            reviews.extend(page_reviews)
            page += 1
    except Exception as e:
        print(f"An error occurred during scraping for {phone_name}: {e}")
    return reviews

# List to hold all reviews from all phones
all_reviews = []

# Loop through each phone and scrape reviews
for phone_name, phone_url in phone_urls.items():
    phone_reviews = scrape_reviews_for_phone(phone_name, phone_url)
    all_reviews.extend(phone_reviews)

# Close the WebDriver after scraping all phones
driver.quit()

# Save all reviews into a single CSV file
if all_reviews:
    df = pd.DataFrame(all_reviews, columns=['Product ID', 'Phone Name', 'Rating', 'Review'])
    df.to_csv(r'E:\GUVI\FINAL PROJECT\combined_flipkart_phone_reviews.csv', index=False, encoding='utf-8')

    print("All reviews saved to 'combined_flipkart_phone_reviews.csv'.")
else:
    print("No reviews to save.")


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,632 B in 3s (1,116 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... D

In [ ]:
import pkg_resources
import sys

# List of packages to check, including Python
packages = [
    "beautifulsoup4",
    "selenium",
    "pandas",
    "textblob",
    "transformers",
    "streamlit",
    "matplotlib",
    "seaborn",
]

# Check Python version
print(f"Python: {sys.version}")

# Check versions of specified packages
for package in packages:
    try:
        version = pkg_resources.get_distribution(package).version
        print(f"{package}: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"{package} is not installed.")


Python: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
beautifulsoup4: 4.13.3
selenium: 4.28.1
pandas: 2.2.2
textblob: 0.19.0
transformers: 4.48.3
streamlit is not installed.
matplotlib: 3.10.0
seaborn: 0.13.2
